##### Building Geoprocessing Tools and Services
* Amanda Vann  


Python Script: https://github.com/UW-GEOG458-Winter2016/avann/blob/gh-pages/Tools/toolboxscript.py         
Toolbox URL: http://qilin.geog.uw.edu:6080/arcgis/rest/services/Whtcrow/Reclassifyscript/GPServer  

Due to the usefulness of python notebook, I did all writing, editing and commenting regarding the scripting process in Ipynb, and then made a new python file and copied the pythyon scripts that I would use for the toolbox to that file.

In [11]:
import sys
sys.path.append(r'C:\Program Files (x86)\ArcGIS\Desktop10.3\bin')
sys.path.append(r'C:\Program Files (x86)\ArcGIS\Desktop10.3\arcpy')
sys.path.append(r'C:\Program Files (x86)\ArcGIS\Desktop10.3\ArcToolbox\Scripts')

# Because Workspace
import arcpy     
import os
arcpy.env.workspace="F:\\UW\\GEOG458 Python\\Lab 2_ToolsService\\workspace\\"  

In [40]:
# Nested Cursor Paramaters
arcpy.env.workspace = r"U:\GEOG458 Python\Workspace\Tools\workspace"

infile = "U:\\GEOG458 Python\\Workspace\\Tools\\workspace\\King.shp"
popdens = "PopDens12"

mytable = r"U:\GEOG458 Python\Workspace\Tools\workspace\ReclassTableExample.dbf"
lower = "lowerbound"
upper = "upperbound"
value = "value"

output = r"U:\GEOG458 Python\Workspace\Tools\workspace\output\test5.shp"
outvalue = "rank"
notfoundvalue = "9999"

I initialy tested the nested cursor paramiters through dirrect links, and after I was able to get the script to work, I changed them to ParameterAsText(1) so as to be recognized by the toolbox. After some issues of not being able to get the toolbox to recognize my script, I sought help and was adviced by the TA to change it to GetParameterAsText(1) instead.

In [2]:
# Nested Cursor Paramaters
infile = GetParameterAsText(0)
popdens = GetParameterAsText(1)

mytable = GetParameterAsText(2)
upper = GetParameterAsText(3) #reordered upper and lower values for ease of reading in the database
lower = GetParameterAsText(4)
value = GetParameterAsText(5)

output = GetParameterAsText(6)
outvalue = "class"
notfoundvalue = GetParameterAsText(7)

I started by running scripts that listed the field content of both 'infile' and 'mytable' so as to make sure that the spelling of the fields being used were correct.  Rather then have to copy and paste the shapefiles in the occurence of errors, I made a copy of the king file for the output that I could delete as I went along if there were issues.

In [41]:
# Copy shapefiles 
arcpy.CopyFeatures_management(infile, output)

<Result 'U:\\GEOG458 Python\\Workspace\\Tools\\workspace\\output\\test5.shp'>

In [42]:
# Add a field
arcpy.AddField_management(output, outvalue, "LONG")

<Result 'U:\\GEOG458 Python\\Workspace\\Tools\\workspace\\output\\test5.shp'>

Following along with results from the previous lab of using nested 
I origionaly used the following script for the nested cursors:

> cursor1 = arcpy.da.UpdateCursor(output, [popdens, outvalue])   
> cursor2 = arcpy.da.UpdateCursor(mytable, [upper, lower, value])   

It claimed to run, but then when I open the shapefile in arcmap to double check that the information was copied correctly, the field row is still all 0's.  
When it was not giving falls runs, it was giving the following error.

> RuntimeError: A column was specified that does not exist. 

I tried a varity of methods with no success and eventualy emailed for help as I could not understand what was wrong with my script.  After taking the proffesors advice of running print scripts for debugging and found that it would not read the row where I had the lesser and greater symbles.  I did several corrections of syntax to try to fix the issues of the script not running.  Despite edits, the script still did not seem to run.   

I eventualy changed it to use a 'with' script, which seemed to work.

In [5]:
with arcpy.da.UpdateCursor(output,[popdens,outvalue]) as cursor1:
    for row1 in cursor1:
        with arcpy.da.SearchCursor(mytable,[upper,lower,value]) as cursor2:
            for row2 in cursor2:
                if row1[0] <= row2[0] and row1[0] >= row2[1]:
                    row1[1] = row2[2]
                    cursor1.updateRow(row1)
                else:
                    row1[1] = novalue
                    cursor1.updateRow(row1)
                # I had attempted a cursor.next() field to get the else to loop when it refused to loop at one point.
                    
print arcpy.GetMessages() 
del cursor1, cursor2, row1, row2 #added so as to get the curors to stop locking the file.

Executing: AddField "U:\GEOG458 Python\Workspace\Tools\workspace\output\tryingscript.shp" rank DOUBLE # # # # NULLABLE NON_REQUIRED #
Start Time: Tue Feb 09 16:35:56 2016
Adding rank to U:\GEOG458 Python\Workspace\Tools\workspace\output\tryingscript.shp...
Succeeded at Tue Feb 09 16:35:58 2016 (Elapsed Time: 2.54 seconds)


After belatedly discovering that the rows were not recognizing the else value, I tried a variaty of different methods but the script would either ignore the else entirely, or replace all fields that were not the the same value as the last row with the null value. 

I attempted a script that would count the rows in mytable, and then once it reached the last row, it would replace all no value fields with the null value number.

>  result = arcpy.GetCount_management(mytable)  
>  count = int(result.getOutput(0))  
>  print(count)  
>  field = 0  

I tried using a mix of counting and setting a loop script to 'else', 'elif' and 'if' but none of them worked.  After speaking to Hardy, he suggested having an entirely separate update cursor that would find the left over values and designated it to the value that the user may want.

In [43]:
with arcpy.da.UpdateCursor(output,[popdens,outvalue]) as cursor1:
    for row1 in cursor1:
        with arcpy.da.SearchCursor(mytable,[upper,lower,value]) as cursor2:
            for row2 in cursor2:
                if row1[0] <= row2[0] and row1[0] >= row2[1]:
                    row1[1] = row2[2]
                    cursor1.updateRow(row1)
        
# edited in attempt to get the cursor to see the null values.        
with arcpy.da.UpdateCursor(output,[popdens,outvalue]) as cursor3: 
    for row3 in cursor3:
        if row3[1] == 0:
            row3[1] = notfoundvalue
            cursor3.updateRow(row3)
                    
print arcpy.GetMessages() 
del cursor1, cursor2, cursor3, row1, row2, row3 #  added so as to get the curors to stop locking the file.

Executing: AddField "U:\GEOG458 Python\Workspace\Tools\workspace\output\test5.shp" rank LONG # # # # NULLABLE NON_REQUIRED #
Start Time: Wed Feb 10 14:23:55 2016
Adding rank to U:\GEOG458 Python\Workspace\Tools\workspace\output\test5.shp...
Succeeded at Wed Feb 10 14:23:57 2016 (Elapsed Time: 1.73 seconds)
